In [3]:
from keplergl import KeplerGl
import pandas as pd

In [4]:
'''
Define paths and parameters
'''
DATA_PATH = './17_18.csv'

In [6]:
# load dataset
DATASET = pd.read_csv(DATA_PATH, sep=';', parse_dates=['Datum'])
print(DATASET.head(5))

       Datum Uhrzeit Kommerzieller Nettoexport[MWh]  \
0 2017-01-01   00:00                          9.516   
1 2017-01-01   01:00                          9.569   
2 2017-01-01   02:00                          9.283   
3 2017-01-01   03:00                          9.379   
4 2017-01-01   04:00                          8.897   

   Niederlande (Export)[MWh] Niederlande (Import)[MWh]  Schweiz (Export)[MWh]  \
0                      1.521                        -2                    1.2   
1                      1.365                        -2                    1.2   
2                      1.329                        -2                    1.2   
3                      1.203                        -2                    1.2   
4                      1.191                        -2                    1.2   

  Schweiz (Import)[MWh]  Dänemark (Export)[MWh] Dänemark (Import)[MWh]  \
0                    -0                     0.0                   -585   
1                    -0           